In [1]:
from abc import ABC, abstractmethod

# ======= CÁC LỚP TRỪU TƯỢNG =======
class BaseProduct(ABC):
    def __init__(self, name, price):
        self.name = name
        self.price = price

    @abstractmethod
    def get_display_price(self):
        pass

class DiscountStrategy(ABC):
    @abstractmethod
    def apply_discount(self, total):
        pass

class PaymentGateway(ABC):
    @abstractmethod
    def process_payment(self, amount):
        pass

class ShippingProvider(ABC):
    @abstractmethod
    def calculate_shipping_fee(self, item):
        pass

    @abstractmethod
    def schedule_shipping(self, item):
        pass

class ReportGenerator(ABC):
    @abstractmethod
    def generate_report(self, data):
        pass

# ======= SẢN PHẨM =======
class PhysicalProduct(BaseProduct):
    def get_display_price(self):
        return self.price

class DigitalProduct(BaseProduct):
    def get_display_price(self):
        return self.price * 0.9  # Giảm 10% mặc định

# ======= CHIẾN LƯỢC GIẢM GIÁ =======
class PercentageDiscount(DiscountStrategy):
    def __init__(self, percent):
        self.percent = percent

    def apply_discount(self, total):
        return total * (1 - self.percent / 100)

# ======= CỔNG THANH TOÁN =======
class PaypalGateway(PaymentGateway):
    def process_payment(self, amount):
        print(f"💳 Thanh toán {amount:.2f} VNĐ qua PayPal.")
        return True

# ======= VẬN CHUYỂN =======
class FastDelivery(ShippingProvider):
    def calculate_shipping_fee(self, item):
        return 10  # Phí cố định

    def schedule_shipping(self, item):
        print(f"🚚 Giao hàng sản phẩm: {item.product.name}")

# ======= KHO =======
class Warehouse:
    def __init__(self):
        self.stock = {}

    def add_product(self, product, quantity):
        self.stock[product] = self.stock.get(product, 0) + quantity

    def update_stock(self, product, quantity):
        if self.stock.get(product, 0) >= quantity:
            self.stock[product] -= quantity
            if self.stock[product] < 3:
                print(f"⚠️ Cảnh báo: {product.name} sắp hết hàng.")
        else:
            raise Exception("Không đủ hàng trong kho!")

    def get_quantity(self, product):
        return self.stock.get(product, 0)

# ======= ĐƠN HÀNG =======
class OrderItem:
    def __init__(self, product, quantity):
        self.product = product
        self.quantity = quantity

    def get_total_price(self):
        return self.product.get_display_price() * self.quantity

class Order:
    def __init__(self, discount_strategy, shipping_provider, payment_gateway):
        self.items = []
        self.discount_strategy = discount_strategy
        self.shipping_provider = shipping_provider
        self.payment_gateway = payment_gateway
        self.total_price = 0

    def add_item(self, product, quantity):
        self.items.append(OrderItem(product, quantity))

    def calculate_total(self):
        subtotal = sum(item.get_total_price() for item in self.items)
        discounted = self.discount_strategy.apply_discount(subtotal)
        shipping_fee = sum(
            self.shipping_provider.calculate_shipping_fee(item)
            for item in self.items if isinstance(item.product, PhysicalProduct)
        )
        self.total_price = discounted + shipping_fee

    def checkout(self):
        self.calculate_total()
        return self.payment_gateway.process_payment(self.total_price)

    def process_order(self, warehouse):
        for item in self.items:
            warehouse.update_stock(item.product, item.quantity)
            if isinstance(item.product, PhysicalProduct):
                self.shipping_provider.schedule_shipping(item)

# ======= BÁO CÁO =======
class InventoryReport(ReportGenerator):
    def generate_report(self, warehouse):
        print("\n📦 BÁO CÁO TỒN KHO:")
        for product, qty in warehouse.stock.items():
            print(f"{product.name}: {qty} sản phẩm")

class SalesReport(ReportGenerator):
    def generate_report(self, orders):
        print("\n💰 BÁO CÁO DOANH THU:")
        for i, order in enumerate(orders, 1):
            print(f"Đơn hàng {i}: {order.total_price:.2f} VNĐ")

# ======= CHẠY CHƯƠNG TRÌNH =======
if __name__ == "__main__":
    # Khởi tạo sản phẩm
    ao_thun = PhysicalProduct("Áo Thun", 100)
    ebook = DigitalProduct("Ebook Python", 50)

    # Tạo kho và thêm sản phẩm
    warehouse = Warehouse()
    warehouse.add_product(ao_thun, 5)
    warehouse.add_product(ebook, 10)

    # Khởi tạo đơn hàng
    discount = PercentageDiscount(10)  # Giảm 10%
    shipping = FastDelivery()
    payment = PaypalGateway()
    order = Order(discount, shipping, payment)

    # Thêm sản phẩm vào đơn hàng
    order.add_item(ao_thun, 2)
    order.add_item(ebook, 1)

    # Thanh toán và xử lý đơn hàng
    if order.checkout():
        order.process_order(warehouse)

    # Tạo báo cáo
    inventory_report = InventoryReport()
    sales_report = SalesReport()

    inventory_report.generate_report(warehouse)
    sales_report.generate_report([order])


💳 Thanh toán 230.50 VNĐ qua PayPal.
🚚 Giao hàng sản phẩm: Áo Thun

📦 BÁO CÁO TỒN KHO:
Áo Thun: 3 sản phẩm
Ebook Python: 9 sản phẩm

💰 BÁO CÁO DOANH THU:
Đơn hàng 1: 230.50 VNĐ
